In [6]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
#  use Image Data Generator to reshape images to avoid overfitting
# 1. / 255 -> scaling
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
# class mode = 'binary' because of the binary outcome. for more outcomes -> category
training_set = train_datagen.flow_from_directory('../datasets/cats_dogs_images/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [5]:
# Image Data Generator for the test set -> rescale only!
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('../datasets/cats_dogs_images/test_set',
    # use same target size and batch size as on training set
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [7]:
cnn = tf.keras.models.Sequential()

2023-04-04 11:03:55.644980: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Add layers

In [9]:
# add convolutional layers
# filters -> number of feature detectors
# kernel size -> dimensions 3x3
# input shape 64, 64 pixels + 3 of rgb channels
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# create pooling
# pool_size -> 2x2 pixels (picks the max value from the feature frame)
# strides -> shift by 2 pixels 
# paddind: 'valid' (no padding/default), 'same' use padding and don't calculate the vslues outside the square)
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# add 2nd convolutional layer with the pool
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# flattening
cnn.add(tf.keras.layers.Flatten())

Connect layers and add an output layer

In [10]:
# bigger number of units since we deal with images 64x64
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
# units = 1 and activation sigmoid (for binary output), softmax for many categories
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Compile and train

In [ ]:
cnn.compile(optimizer='Adam', )